# Financial Instruments - Homework 2

## 1. Exploiting an Apparent Arbitrage Opportunity

**1.1 Set up the arbitrage trade on Oct 1, 2008 for the 1-year forward.**

From previous analysis, the 1-year forward rate $F_{mkt}$ is higher than the theoretical rate $F_{theo}$.
Strategy: **Sell Forward, Buy Synthetic**.

**Trade Steps:**
1. Sell Forward: Contract to sell 1 EUR at $T=1$ for $F_{0}$.
2. Buy Synthetic:
   - Borrow USD: $S_0 \cdot e^{-r_{EUR} T}$.
   - Buy EUR: $e^{-r_{EUR} T}$.
   - Invest EUR: At $r_{EUR}$ to grow to 1 EUR at $T$.

In [ ]:
import pandas as pd
import numpy as np

# Data Loading
file_path = "Assignments/Assignment 2/DataHW2_2024.xls"
df_arb = pd.read_excel(file_path, sheet_name='ForwardArbitrage', header=None)

def get_data(row_idx):
    row = df_arb.iloc[row_idx]
    return {
        'Date': row[0],
        'T_rem': row[1],
        'Spot': row[2],
        'Fwd': row[3],
        'r_US': row[4]/100.0,
        'r_EU': row[5]/100.0
    }

# Indices based on file structure
d0 = get_data(4) # Oct 1 2008
d1 = get_data(5) # Apr 1 2009

# Params
T = 1.0
S0 = d0['Spot']
F0 = d0['Fwd']
rUS_0_cc = np.log(1 + d0['r_US'] * T) / T
rEU_0_cc = np.log(1 + d0['r_EU'] * T) / T

# Unwind at t=0.5
t = 0.5
rem_T = 0.5
St = d1['Spot']
Ft = d1['Fwd']
rUS_t_cc = np.log(1 + d1['r_US'] * rem_T) / rem_T
rEU_t_cc = np.log(1 + d1['r_EU'] * rem_T) / rem_T

# Value Short Forward
val_short_fwd = (F0 - Ft) * np.exp(-rUS_t_cc * rem_T)

# Value Synthetic Long
val_long_leg = St * np.exp(-rEU_t_cc * rem_T)
usd_borrow_0 = S0 * np.exp(-rEU_0_cc * T)
N_debt = usd_borrow_0 * np.exp(rUS_0_cc * T)
val_short_leg = N_debt * np.exp(-rUS_t_cc * rem_T)

val_synthetic = val_long_leg - val_short_leg
total_pl = val_short_fwd + val_synthetic

print(f"Value Short Forward: {val_short_fwd}")
print(f"Value Synthetic: {val_synthetic}")
print(f"Total P&L: {total_pl}")

## 3. Hedging with Futures: Southwest

**3.1 Strategy**
Long FEB, MAR, APR futures to hedge Jan, Feb, Mar fuel consumption.

In [ ]:
df_raw = pd.read_excel(file_path, sheet_name=0, header=None)
q3_1 = df_raw.iloc[4:60, [0, 1, 2, 3, 6]].copy()
q3_1.columns = ['Date', 'FEB.08', 'MAR.08', 'APR.08', 'FuelPrice']
q3_1['Date'] = pd.to_datetime(q3_1['Date'], errors='coerce')
q3_1 = q3_1.dropna(subset=['Date'])

gallons_needed_monthly = 1511e6 / 12
hedge_ratio = 0.75
contract_size = 1000 * 42
n_contracts = round((gallons_needed_monthly * hedge_ratio) / contract_size)

prices_0 = {'FEB': 95.98, 'MAR': 95.78, 'APR': 95.24}
results = []

for month, col in zip(['FEB', 'MAR', 'APR'], ['FEB.08', 'MAR.08', 'APR.08']):
    series = q3_1[['Date', col]].dropna()
    if series.empty: continue
    date_T = series.iloc[-1]['Date']
    price_T = series.iloc[-1][col]
    
    fuel_row = q3_1[q3_1['Date'] == date_T]
    if fuel_row.empty: continue
    fuel_price_T = fuel_row.iloc[0]['FuelPrice']
    
    gain = (price_T - prices_0[month]) * 1000 * n_contracts
    cost = gallons_needed_monthly * fuel_price_T - gain
    implicit = cost / gallons_needed_monthly
    
    results.append({'Month': month, 'Gain': gain, 'Implicit': implicit})

pd.DataFrame(results)